## **Промежуточная аттестация 3**

### Считываем данные

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv('https://dc-edu.itmo.ru/assets/courseware/v1/431a30c4a3a2b3f265f8cce1a809aa0f/asset-v1:ITMO+bonus_track_methods+2023+type@asset+block/pulsar_stars_new.csv')
df

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TG
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0
...,...,...,...,...,...,...,...,...,...
17893,136.429688,59.847421,-0.187846,-0.738123,1.296823,12.166062,15.450260,285.931022,0
17894,122.554688,49.485605,0.127978,0.323061,16.409699,44.626893,2.945244,8.297092,0
17895,119.335938,59.935939,0.159363,-0.743025,21.430602,58.872000,2.499517,4.595173,0
17896,114.507812,53.902400,0.201161,-0.024789,1.946488,13.381731,10.007967,134.238910,0


## Подготовка данных

Сформируем выборку на основании следующих критериев:

* Все строки, где `TG` = 0 и `MIP` в диапазоне `[52.40625, 175.328125]`

* Все строки, где `TG` = 1 и `MIP` в диапазоне `[10.015625, 120.5546875]`

In [3]:
df = df[((df['TG'] == 0) & (df['MIP'] >= 52.40625) & (df['MIP'] <= 175.328125)) |
        ((df['TG'] == 1) & (df['MIP'] >= 10.015625) & (df['MIP'] <= 120.5546875))]

Определим выборочное среднее для столбца `STDIP`

In [4]:
round(df['STDIP'].mean(), 3)

46.575

Отсортируем данные по возрастанию столбца `SIP`

In [5]:
df = df.sort_values(by='SIP')
df.head()

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TG
2137,135.859375,81.503042,0.023921,-1.791886,3.655518,19.638924,7.228962,63.401192,0
5291,101.242188,91.808628,0.290115,-1.781888,195.795987,63.624015,-1.852570,2.544999,0
3509,113.265625,98.778911,0.179404,-1.764717,145.361204,64.773789,-0.154935,-1.374633,0
10625,141.757812,91.206475,-0.056413,-1.755332,55.243311,75.793948,1.047847,-0.255939,0
370,107.539062,86.951396,-0.008928,-1.676724,211.948997,54.010310,-2.556795,5.768458,0


Разделим набор данных на тренировочную и тестовую выборки с помощью `train_test_split()` в соотношении `80:20` c параметром `random_state = 21`. Используем стратификацию по колонке отклика.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='TG'), df['TG'], test_size=0.2, random_state=21, stratify=df['TG'])

print('Shapes', '    X', '       y')
print('train', X_train.shape, y_train.shape)
print('test ', X_test.shape, y_test.shape)

Shapes     X        y
train (14238, 8) (14238,)
test  (3560, 8) (3560,)


Определим выборочное среднее для столбца `MC` из тренировочной выборки

In [7]:
round(X_train['MC'].mean(), 3)

12.091

Выполним преобразование линейной нормировки для тренировочного набора данных и примените его для тестовых данных

In [8]:
scale = MinMaxScaler().fit(X_train)

X_train = pd.DataFrame(scale.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scale.transform(X_test), columns=X_test.columns)

Определим выборочное среднее для столбца `SC` из тренировочной выборки (после нормировки)

In [9]:
round(X_train['SC'].mean(), 3)

0.09

### Обучаем `LogisticRegression`

Обучим модель `LogisticRegression(random_state = 21)` на тренировочных данных

In [10]:
lr = LogisticRegression(random_state = 21).fit(X_train, y_train)

Оценим модель на тестовых данных. Составим матрицу ошибок.

In [11]:
prediction = lr.predict(X_test)

conf_matrix = confusion_matrix(y_test, prediction)
print('tn, fp, fn, tp')
print(conf_matrix.ravel())

tn, fp, fn, tp
[3230   16   71  243]


Вычислим `f1_score` для тестового набора данных

In [12]:
round(f1_score(y_test, prediction), 3)

0.848

### Обучаем `KNeighborsClassifier`

Обучаем модель `KNeighborsClassifier(n_neighbors=3)` на тренировочных данных

In [13]:
knn = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)

Оценим модель на тестовых данных. Составим матрицу ошибок.

In [14]:
prediction = knn.predict(X_test)

conf_matrix = confusion_matrix(y_test, prediction)
print('tn, fp, fn, tp')
print(conf_matrix.ravel())

tn, fp, fn, tp
[3222   24   48  266]


Вычислим `f1_score` для тестового набора данных

In [15]:
round(f1_score(y_test, prediction), 3)

0.881